## 从雅虎财经网站获取股票数据

In [2]:
import urllib
import os
import datetime
import pandas as pd
import numpy as np

In [2]:
# 下载整个股票数据
def retrive_stock_data(stockid, folder):
    print('downloading %s to %s' % (stockid, folder))
    url = 'http://table.finance.yahoo.com/table.csv?s=%s' % (stockid)
    fname = os.path.join(folder, '%s.csv' % stockid.split('.')[0])
    if not os.path.isdir(folder):
        os.mkdir(folder)
    urllib.urlretrieve(url, fname)

In [24]:
retrive_stock_data('600690.ss', 'yahoo-data')

downloading 600690.ss to yahoo-data


In [3]:
# 更新股票数据，如果不存在，则下载。如果存在，则只更新最近日期的数据
def update_stock_data(stockid, folder):
    fname = os.path.join(folder, '%s.csv' % stockid.split('.')[0])
    if not os.path.exists(fname):
        retrive_stock_data(stockid, folder)
        return
    
    data = pd.read_csv(fname, index_col='Date', parse_dates=True)
    
    last_date = data.iloc[0:1].index.tolist()[0]
    today = pd.Timestamp(datetime.date.today())
    if today - last_date < pd.Timedelta(days=2):
        print('Nothing to update. %s last date is %s.' % (stockid, last_date))
        return
    
    print('updatting %s to from %s to %s' % (stockid, last_date.date(), today.date()))
    query = [
        ('a', last_date.month - 1),
        ('b', last_date.day),
        ('c', last_date.year),
        ('d', today.month - 1),
        ('e', today.day),
        ('f', today.year),
        ('s', stockid),
    ]
    url = 'http://table.finance.yahoo.com/table.csv?%s' % urllib.urlencode(query)
    temp_file = fname + '.tmp'
    urllib.urlretrieve(url, temp_file)
    update_data = pd.read_csv(temp_file, index_col='Date', parse_dates=True)
    data = data.append(update_data)
    data.sort_index(ascending=False, inplace=True)
    data.to_csv(fname, mode='w')
    os.unlink(temp_file)

In [4]:
update_stock_data('600690.ss', 'yahoo-data')

downloading 600690.ss to yahoo-data


IOError: [Errno socket error] [Errno 10061] No connection could be made because the target machine actively refused it

### 获取所有的股票列表

In [4]:
sh = pd.read_csv('SH.txt', header=None, names=['name', 'id'], dtype={'id': np.string0})
sh['postfix']='.ss'
sh.iloc[0]['id'].strip()

'600000'

In [20]:
sz = pd.read_csv('SZ.txt', header=None, names=['name', 'id'], dtype={'id': np.string0})
sz['postfix']='.sz'
sz

,name,id,postfix
0,平安银行,000001,.sz
1,万科A,000002,.sz
2,PT金田A,000003,.sz
3,国农科技,000004,.sz
4,世纪星源,000005,.sz
5,深振业A,000006,.sz
6,零七股份,000007,.sz
7,神州高铁,000008,.sz
8,中国宝安,000009,.sz
9,深华新,000010,.sz


In [32]:
slist = pd.concat([sh, sz])

In [9]:
url = 'http://table.finance.yahoo.com/table.csv?a=10&b=28&c=2015&d=10&e=28&f=2015&s=600001.ss'
[fname, header] = urllib.urlretrieve(url, 'data.csv')
print fname
header

data.csv


<httplib.HTTPMessage instance at 0x10954e9e0>

In [4]:
pd.read_csv?